#CM4709 Computer Vision
#Lab 07 Image Segmentation

##Aims
. Use Mask R-CNN to perform image segmentation.

##Mounting GoogleDrive to Colab Runtime

As usual, you will need some testing images. It is faster to upload them to GoogleDrive and then mount it to your Colab runtime.

In the following code, I use the path `/content/drive` as the mounting point as Colab also has a convenient button mounting to this path.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Testing Images

You are advised to organise your files in GoogleDrive. e.g. Create a `Lab07` folder in GoogleDrive to store the notebook file of Lab 07, and a `Lab07/data` folder for the images.

1. Download the testing images in Moodle. Optionally, use your own testing images.
1. Upload testing images to your GoogleDrive space.

The following code shows a testing image using OpenCV.
* Note: Modify the filename and image path accordingly.


In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

#image filename and path
#
filename='test.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#show image
#
img=cv.imread(imageFile)
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

##Install PixelLib

[PixelLib](https://github.com/ayoolaolafenwa/PixelLib) provides some handy libraries to do image segmentation. Pixellib uses Tensorflow.

The following shell commands install Pixellib and also Tensorflow version 2.5.0 which works with PixelLib.

Note:
* The current version of Tensorflow in Colan is 2.12.0. This version does not work with PixelLib.
* After trying many times, I found that Tensorflow 2.5.0 still works. Online resources said that PixelLib works with Tensorflow 2.4.0 but unluckily this version is not available in Colab anymore.


In [ ]:
!pip install pixellib --upgrade
!pip install tensorflow==2.5.0


##The Pre-trained Model/Weights File

To run Mask R-CNN, you need the pre-trained modle's weights file.
Mask R-CNN has been trained with the COCO dataset.

The following shell command downloads the weights file into the default folder of your Colab runtime:

In [ ]:
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5


Every time your Colab runtime is re-connected, files stored inside will be gone.
Alternatively, you may want to download the weights file into your GoogleDrive space so that you don't need to re-download everytime.
If you choose to do this:

1. Download the weights file here: [https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5](https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5)
1. Upload it to your GoogleDrive. e.g. in the `Lab07/data` folder.
1. In later references to the weights file, modify the path accordingly.


##Creating the Model

You are now ready to create a Mask R-CNN model to do image segmentation.
In the following code:
* Modify the path the image if needed.
* Modify the path to the weights file if needed.
* The function `segmentImage(...)` takes a parameter `output_image_name`. The image segmentation output will be written to this file. Modify this if you need to.

In [ ]:
import pixellib
from pixellib.instance import instance_segmentation

#image filename and path
#
filename='test.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#COCO weights file
#
weightFile=imagePath+'/mask_rcnn_coco.h5'

#output of segementation
#
outputFilename='output.jpg'
outputFile=imagePath+'/'+outputFilename

segment_image = instance_segmentation()
segment_image.load_model(weightFile)
segment_image.segmentImage(imageFile, output_image_name=outputFile)

##Seeing the Segmentation Result

Mask R-CNN outputs the result to the image file you specified.
The following OpenCV code simply displays the output:

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

#output of segementation
#
outputFilename='output.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
outputFile=imagePath+'/'+outputFilename

#show segementation result
#
img=cv.imread(outputFile)
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))


##Other Functions

[PixelLib](https://pixellib.readthedocs.io/en/latest/) has many handy functions built-in.
You can see the documentation [here](https://pixellib.readthedocs.io/en/latest/Image_instance.html).
For example, you can show the bounding boxes of the detected segments:


In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import pixellib
from pixellib.instance import instance_segmentation

#image filename and path
#
filename='test2.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#COCO weights file
#
weightFile=imagePath+'/mask_rcnn_coco.h5'

#output of segementation
#
outputFilename='output.jpg'
outputFile=imagePath+'/'+outputFilename

segment_image = instance_segmentation()
segment_image.load_model(weightFile)
segment_image.segmentImage(imageFile, output_image_name=outputFile,show_bboxes=True)

#show segementation result
#
img=cv.imread(outputFile)
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))


You can also extract and save the detected segments:

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import pixellib
from pixellib.instance import instance_segmentation

#image filename and path
#
filename='test2.jpg'
imagePath='/content/drive/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#COCO weights file
#
weightFile=imagePath+'/mask_rcnn_coco.h5'

#output of segementation
#
outputFilename='output.jpg'
outputFile=imagePath+'/'+outputFilename

segment_image = instance_segmentation()
segment_image.load_model(weightFile)
segment_image.segmentImage(imageFile, output_image_name=outputFile,extract_segmented_objects=True,save_extracted_objects=True)


Extracted segments are saved to the home of your Colab runtime with filenames `segmented_object_...`.
The following code shows all of them:

In [ ]:
import os
import fnmatch

segmentPath='.'
pattern='segmented_object_*.jpg'
segmentFiles=[]

#collect segment files full paths
#
allFiles=os.listdir(segmentPath)
for file in allFiles:
  if fnmatch.fnmatch(file,pattern):
    segmentFiles.append(segmentPath+'/'+file)

total=len(segmentFiles) #no. of files
print('No. of files: ',total)

colCount=10
rowCount=int(total/colCount)+1
plt.figure(figsize=(colCount*1.5,rowCount*1.5))
i=1

for file in segmentFiles:
  img=cv.imread(file)
  cvtImg=cv.cvtColor(img, cv.COLOR_BGR2RGB)
  plt.subplot(rowCount,colCount,i)
  plt.imshow(cvtImg)
  i=i+1  

If you need to remove all `segemented_object_*.jpg` files between different runs, the following code will be handy:

In [44]:
#remove all extracted segment files
#
import os
import fnmatch

segmentPath='.'
pattern='segmented_object_*.jpg'

#collect segment files full paths
#
allFiles=os.listdir(segmentPath)
for file in allFiles:
  if fnmatch.fnmatch(file,pattern):
    os.remove(segmentPath+'/'+file)  